In [14]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from  pyspark.sql import SQLContext
from pyspark.sql.types import *

import shutil
import os

In [15]:
spark = SparkSession.builder \
             .master("local") \
             .appName("stream to parquet") \
             .config("spark.some.config.option", "some-value") \
            .getOrCreate()

In [16]:
dataSchema = StructType([StructField('Arrival_Time',LongType(),True),
                             StructField('Creation_Time',LongType(),True),
                             StructField('Device',StringType(),True),
                             StructField('Index',LongType(),True),
                             StructField('Model',StringType(),True),
                             StructField('User',StringType(),True),
                             StructField('gt',StringType(),True),
                             StructField('x',DoubleType(),True),
                             StructField('y',DoubleType(),True),
                             StructField('z',DoubleType(),True)
                        ])

In [17]:
#Global
CHECKPOINT_DIR="checkpoint_parquet_dir"

In [18]:
def clean_dirs():
    shutil.rmtree(CHECKPOINT_DIR,ignore_errors=True)
    os.mkdir(CHECKPOINT_DIR)
clean_dirs()

In [19]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.json( '/home/henry/projects/Spark-The-Definitive-Guide/data/activity-data_small')
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [20]:
streaming.registerTempTable("my_table")

In [21]:
"""spark.sql("select * from my_table").writeStream.format("parquet")\
.option("checkpointLocation", CHECKPOINT_DIR).outputMode("append")\
.option("path", "stream_dir_parquet1").start()"""

'spark.sql("select * from my_table").writeStream.format("parquet").option("checkpointLocation", CHECKPOINT_DIR).outputMode("append").option("path", "stream_dir_parquet1").start()'

In [22]:
spark.sql("select * from my_table").writeStream.format("parquet")\
.option("path", "stream_dir_parquet1").start()

AnalysisException:  checkpointLocation must be specified either through option("checkpointLocation", ...) or SparkSession.conf.set("spark.sql.streaming.checkpointLocation", ...)        

In [9]:
import os
p = '/home/henry/projects/big-data/big_data/stream_dir_parquet1/'
os.path.isdir(p)

True

In [10]:
streaming2 = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.parquet('/home/henry/projects/big-data/big_data/stream_dir_parquet1')


In [11]:
del(streaming2)

In [12]:
df = spark.read.parquet('/home/henry/projects/big-data/big_data/stream_dir_parquet1')
df.createOrReplaceTempView("my_table2")
spark.sql("SELECT count(*) from my_table2").show()


+--------+
|count(1)|
+--------+
|  546084|
+--------+



In [ ]:
"""If you add a filel to /home/henry/projects/Spark-The-Definitive-Guide/data/activity-data_small
and then run the above cell, you will get a different count"""

In [ ]:
gg